In [9]:
import camelot
import fitz

In [12]:
file_path = 'IOE_microprocessor_chapter_1.pdf'

pdf = fitz.open(file_path)
print(len(pdf))

pages_with_table =[]
table_coordinates = []
table_data = []

dict_tables = {}


for i in range(1,len(pdf)):
    tables = camelot.read_pdf(file_path,pages=f"{i}")
    if(tables):
        pages_with_table.append(i)
        
for i in pages_with_table:
    tables = camelot.read_pdf(file_path,pages=f"{i}")
    tables_of_a_page = []
    
    for table in tables:
        tables_of_a_page.append(table._bbox)
        table_data.append(table.df)
    dict_tables[i] = tables_of_a_page

#Now I just need to remove the tables using the coordinates and fitz.Rect
dict_tables
    

15


{1: [(76.08, 390.84156316267797, 206.64, 514.6440472584065)],
 3: [(74.88, 580.6240533171766, 208.32, 699.6279915177219),
  (72.96, 276.3962435625568, 186.0, 430.42956679794)],
 5: [(72.96, 510.56528324750076, 405.12, 639.6461678279309)],
 9: [(66.48, 396.35989094213875, 426.96, 720.0218115722508)]}

In [13]:
#remove each table from the pdf by using the coordinates of table in dict_tables

for num_page, tables in dict_tables.items():
    page = pdf[num_page -1]
    page_height = page.rect.height

    for x1, y1, x2, y2 in tables:
        # Convert Camelot bbox to fitz coordinates (top-left origin)
        y1_fitz = page_height - y1
        y2_fitz = page_height - y2
        rect = fitz.Rect(x1, y2_fitz, x2, y1_fitz)

        # Redact the region (white fill)
        page.add_redact_annot(rect, fill=(1, 1, 1))

    # Apply all redactions after adding them
    page.apply_redactions()

# Save result
out_path = 'table_deleted.pdf'
pdf.save(out_path)        

In [ ]:
p